In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

In [ ]:
# Set the path to the dataset directory
dataset_path = os.path.join('/content/drive/MyDrive/Capstone Project/Batik_DataSet_V2')

img_size = 150
batch_size = 80

In [ ]:
# Data augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.3,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2)

In [ ]:
# Data normalization for the validation set
val_datagen = ImageDataGenerator(
    rescale=1./255)

In [ ]:
# Load and preprocess the training set
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_size, img_size),
    shuffle=True,
    class_mode='categorical',
    subset='training')

# Load and preprocess the validation set
val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_size, img_size),
    shuffle=True,
    class_mode='categorical',
    subset='validation')

Found 723 images belonging to 20 classes.
Found 175 images belonging to 20 classes.


In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
  
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
# Load the pre-trained InceptionV3 model without the top layers
base_model = InceptionV3(weights=None, include_top=False, input_shape=(150, 150, 3))
base_model.load_weights(local_weights_file)
last_layer = base_model.get_layer('mixed7')
last_output = last_layer.output

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

--2023-06-01 17:02:54--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.31.128, 173.194.213.128, 172.253.123.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.31.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   140MB/s    in 0.6s    

2023-06-01 17:02:55 (140 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [ ]:
# Build the model architecture by adding the pre-trained base model and the top layers
model = tf.keras.models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(20, activation='softmax')
])
# Print the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 3, 3, 2048)        21802784  
                                                                 
 flatten_1 (Flatten)         (None, 18432)             0         
                                                                 
 dense_2 (Dense)             (None, 1024)              18875392  
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0         
                                                                 
 dense_3 (Dense)             (None, 20)                20500     
                                                                 
Total params: 40,698,676
Trainable params: 18,895,892
Non-trainable params: 21,802,784
_________________________________________________________________


In [ ]:
# Set the training parameters
model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(learning_rate=0.0001), 
              metrics=['accuracy'])

In [ ]:
# Train the model using the training generator and validate it using the validation generator
history = model.fit(train_generator,
                    epochs=100,
                    steps_per_epoch=20,
                    validation_data=val_generator,
                    verbose=1,
                    validation_steps=50)

Epoch 1/100
9/9 [==============================] - 103s 11s/step - loss: 5.6366 - accuracy: 0.0729 - val_loss: 5.2594 - val_accuracy: 0.1250
Epoch 2/100
9/9 [==============================] - 56s 6s/step - loss: 3.8870 - accuracy: 0.1562 - val_loss: 3.4951 - val_accuracy: 0.1875
Epoch 3/100
9/9 [==============================] - 46s 5s/step - loss: 3.2939 - accuracy: 0.1771 - val_loss: 2.8562 - val_accuracy: 0.1719
Epoch 4/100
9/9 [==============================] - 38s 4s/step - loss: 2.5094 - accuracy: 0.2812 - val_loss: 2.7155 - val_accuracy: 0.2031
Epoch 5/100
9/9 [==============================] - 36s 4s/step - loss: 2.4412 - accuracy: 0.2764 - val_loss: 2.5072 - val_accuracy: 0.3438
Epoch 6/100
9/9 [==============================] - 32s 4s/step - loss: 2.3149 - accuracy: 0.3382 - val_loss: 2.8172 - val_accuracy: 0.1719
Epoch 7/100
9/9 [==============================] - 35s 4s/step - loss: 2.3830 - accuracy: 0.3091 - val_loss: 2.6633 - val_accuracy: 0.2656
Epoch 8/100
9/9 [========

KeyboardInterrupt: ignored